In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


file_path = '/content/drive/MyDrive/train_data.csv' 
df = pd.read_csv(file_path)

print("Dataset Size:", df.shape)
print("\nOriginal Stay Column Values:")
print(df['Stay'].unique())

# 0-10 days -> 0 (short stay), others -> 1 (long stay)
def classify_stay(stay_duration):
    if stay_duration == '0-10':
        return 0
    else:
        return 1

# new column
df['Target'] = df['Stay'].apply(classify_stay)

# class balance
print("\nClass Distribution")
counts = df['Target'].value_counts()
percentages = df['Target'].value_counts(normalize=True) * 100

print(f"Class 1 (Long Stay): {counts[1]}, (%{percentages[1]:.1f})")
print(f"Class 0 (Short Stay):  {counts[0]}, (%{percentages[0]:.1f})")

# visualize
plt.figure(figsize=(6,4))
sns.countplot(x='Target', data=df)
plt.title('Short (0) vs Long (1) Stay Distribution')
plt.show()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


df['Bed Grade'].fillna(df['Bed Grade'].mode()[0], inplace=True)
df['City_Code_Patient'].fillna(df['City_Code_Patient'].mode()[0], inplace=True)


cols_to_drop = ['case_id', 'patientid', 'Stay']
df_clean = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


le = LabelEncoder()
categorical_cols = df_clean.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_clean[col] = le.fit_transform(df_clean[col])


df_majority = df_clean[df_clean.Target == 1] 
df_minority = df_clean[df_clean.Target == 0] 


df_majority_downsampled = resample(df_majority, 
                                 replace=False,    
                                 n_samples=len(df_minority), 
                                 random_state=42) 


df_balanced = pd.concat([df_majority_downsampled, df_minority])




X = df_balanced.drop('Target', axis=1)
y = df_balanced['Target']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nTraining Model...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)

print("\n--- MODEL ---")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


plt.figure(figsize=(5,4))
sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (0: Short, 1: Long Stay)')
plt.ylabel('Actual')
plt.xlabel('Predictied')
plt.show()

In [ ]:
import numpy as np

def predict_final_system():
    print("\n" + "="*60)
    print("CLINICAL PREDICTION SYSTEM (Final Version)")
    print("="*60)

    try:
        
        print("\n--- VITALS & TRIAGE ---")
        bp = int(input("Systolic Blood Pressure (e.g., 120): "))
        pulse = int(input("Heart Rate (e.g., 80): "))
        
        
        severity_val = 1 
        severity_desc = "Minor"
        
        if bp > 160 or pulse > 110:
            severity_val = 0 
            severity_desc = "Extreme (Critical)"
            print("ALERT: Critical Vitals! Severity set to 'Extreme'.")
        elif bp > 140 or pulse > 100:
            severity_val = 2 
            severity_desc = "Moderate"
            print("NOTE: Elevated Vitals. Severity set to 'Moderate'.")
        else:
            print("NOTE: Vitals Stable. Severity set to 'Minor'.")

        
        print("\n--- ADMISSION TYPE ---")
        print("1: Emergency")
        print("2: Trauma")
        print("3: Urgent")
        adm_choice = int(input("Select Type (1-3): "))
        adm_map = {1: 0, 2: 1, 3: 2}
        selected_adm = adm_map.get(adm_choice, 2)

        
        print("\n--- DEPARTMENT ---")
        dept_names = ['Anesthesia', 'Gynecology', 'Radiotherapy', 'Surgery', 'TB & Chest disease']
        for i, name in enumerate(dept_names, 1):
            print(f"{i}: {name}")
        dept_choice = int(input("Select Department (1-5): "))
        selected_dept = dept_choice - 1 

        
        print("\n--- PATIENT HISTORY ---")
        
        
        print("-" * 45)
        print("AGE REFERENCE TABLE:")
        print("0: 0-10   | 1: 11-20  | 2: 21-30  | 3: 31-40")
        print("4: 41-50  | 5: 51-60  | 6: 61-70  | 7: 71-80")
        print("8: 81-90  | 9: 90-100")
        print("-" * 45)
        
        age = int(input("Enter Age Code (0-9): "))
        deposit = float(input("Admission Deposit (e.g., 4000): "))
        visitors = int(input("Visitors with Patient (e.g., 2): "))

       
        new_patient = X_train.mean().values.reshape(1, -1)
        cols = list(X.columns)
        
        
        new_patient[0, cols.index('Admission_Deposit')] = deposit
        new_patient[0, cols.index('Visitors with Patient')] = visitors
        new_patient[0, cols.index('Age')] = age
        
        if 'Type of Admission' in cols:
            new_patient[0, cols.index('Type of Admission')] = selected_adm
        if 'Department' in cols:
            new_patient[0, cols.index('Department')] = selected_dept
        if 'Severity of Illness' in cols:
            new_patient[0, cols.index('Severity of Illness')] = severity_val

        prediction = model.predict(new_patient)[0]
        probs = model.predict_proba(new_patient)[0]

        print("\n" + "*"*45)
        print("FINAL PREDICTION RESULT")
        print("*"*45)
        print(f"Context: {dept_names[selected_dept]} | Age Code: {age} | Severity: {severity_desc}")
        
        if prediction == 0:
            conf = probs[0] * 100
            print(f"\nPREDICTION: SHORT STAY (0-10 Days)")
            print(f"Confidence: {conf:.1f}%")
        else:
            conf = probs[1] * 100
            print(f"\nPREDICTION: LONG STAY (>10 Days)")
            print(f"Confidence: {conf:.1f}%")
            
            if age >= 7: 
                print("Reason: Advanced age significantly increases stay duration.")
            if severity_val == 0:
                print("Reason: Critical vitals require intensive care.")
            if dept_choice == 4:
                print("Reason: Surgery implies recovery period.")

    except Exception as e:
        print("Error:", e)

predict_final_system()

In [ ]:
import joblib


joblib.dump(model, 'hospital_model.pkl')
columns_list = list(X.columns)
joblib.dump(columns_list, 'model_columns.pkl')


from google.colab import files
files.download('hospital_model.pkl')
files.download('model_columns.pkl')

In [ ]:
import pandas as pd
import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample


try:
    df = pd.read_csv('/content/drive/MyDrive/train_data.csv')
    print("Loaded")
except:
    print("ERROR")


df['Target'] = df['Stay'].apply(lambda x: 0 if x == '0-10' else 1)
mask_critical = df['Severity of Illness'] == 'Extreme'
df.loc[mask_critical, 'Target'] = 1 



df['Bed Grade'].fillna(df['Bed Grade'].mode()[0], inplace=True)
df['City_Code_Patient'].fillna(df['City_Code_Patient'].mode()[0], inplace=True)
df_clean = df.drop(columns=['case_id', 'patientid', 'Stay'])


le = LabelEncoder()
for col in df_clean.select_dtypes(include=['object']).columns:
    df_clean[col] = le.fit_transform(df_clean[col])


df_majority = df_clean[df_clean.Target == 1]
df_minority = df_clean[df_clean.Target == 0]
df_majority_downsampled = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_downsampled, df_minority])

X = df_balanced.drop('Target', axis=1)
y = df_balanced['Target']

model = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
model.fit(X, y)

joblib.dump(model, 'hospital_model.pkl')
joblib.dump(list(X.columns), 'model_columns.pkl')

